[View in Colaboratory](https://colab.research.google.com/github/Alfo5123/DisRecGen/blob/master/VAE_baseline_CIFAR10_collab.ipynb)

# <center>VAE-baseline for CIFAR10 </center>

## Load Data

In [1]:
!pip3 install --upgrade torch torchvision

Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (0.4.0)
Requirement already up-to-date: torchvision in /usr/local/lib/python3.6/dist-packages (0.2.1)
Requirement not upgraded as not directly required: six in /usr/local/lib/python3.6/dist-packages (from torchvision) (1.11.0)
Requirement not upgraded as not directly required: pillow>=4.1.1 in /usr/local/lib/python3.6/dist-packages (from torchvision) (5.1.0)
Requirement not upgraded as not directly required: numpy in /usr/local/lib/python3.6/dist-packages (from torchvision) (1.14.3)


In [0]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from IPython.display import Image
from google.colab import files


In [3]:
#Set random seed 
torch.manual_seed(512)

In [4]:
#Get the CIFAR10 train images 
cifar = datasets.CIFAR10('./data/cifar/', train = True, download = True)

Files already downloaded and verified


In [0]:
# Organize training data in batches, 
# normalize them to have values between [-1, 1] (?)

train_images = torch.utils.data.DataLoader ( datasets.CIFAR10('./data/cifar/', train = True, download=False,
                               transform=transforms.Compose([
                               #transforms.Resize(64), 
                               #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                               transforms.ToTensor(),])) , 
                               batch_size = 32, shuffle = True)

## Model

We will use the arquitecture suggested by [Radford et al](https://arxiv.org/abs/1511.06434) for both the encoder and decoder. With convolutional layers in the encoder and fractionally-strided  convolutions  in  the  decoder.   In  each convolutional layer in the encoder we double the number of filters present in the previous layer and use a convolutional stride of 2.  In each convolutional layer in the decoder we use a fractional stride of 2 and halve the number of filters on each layer.

In [0]:
class VAE(nn.Module):

    def __init__(self, image_size ,  hidden_dim , encoding_dim):
        
        super(VAE, self).__init__()
        
        self.encoding_dim = encoding_dim
        self.image_size = image_size
        self.hidden_dim = hidden_dim 
        
        # Decoder - Fractional strided convolutional layers
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 1, 0, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, 4, 2, 1, bias = False),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 3, 4, 2, 1, bias = False),
            nn.Sigmoid() # nn.Tanh()  
        )
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(32, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 256, 4, 2, 0, bias = False),
            nn.Sigmoid()
        )
        
        # Fully-connected layers
        self.fc1 = nn.Linear(256, self.hidden_dim)
        self.fc21 = nn.Linear(self.hidden_dim, self.encoding_dim)
        self.fc22 = nn.Linear(self.hidden_dim, self.encoding_dim)
        self.fc3 = nn.Linear(self.encoding_dim, self.hidden_dim)
        self.fc4 = nn.Linear(self.hidden_dim, 256)
    
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        h4 = F.sigmoid(self.fc4(h3))
        return self.decoder( h4.view(z.size(0),-1,1,1) ) 

        
    def forward(self, x):
        
        # Encode 
        encoded = F.relu(self.fc1( self.encoder(x).view(x.size(0), -1) ) )
        
        #Obtain mu and logvar
        mu = self.fc21( encoded )
        logvar = self.fc22 ( encoded )
        
        #Reparametrization trick
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        z = eps.mul(std).add_(mu)
        
        # Decode 
        decoded = self.decode(z)

        # return decoded, mu, logvar
        return decoded, mu , logvar

    
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, size_average=False)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [0]:
#Define model
model = VAE(32, 128, 32).cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3, amsgrad=True)

#Train model
def train(epoch):
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_images):
        data = Variable(data).cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_images.dataset),
                100. * batch_idx / len(train_images),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_images.dataset)))

In [8]:
num_epochs = 100
for epoch in range(1, num_epochs):
    train(epoch)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2137.845703
Train Epoch: 1 [1600/50000 (3%)]	Loss: 2096.791748
Train Epoch: 1 [3200/50000 (6%)]	Loss: 2016.400513
Train Epoch: 1 [4800/50000 (10%)]	Loss: 2011.956543
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2010.365723
Train Epoch: 1 [8000/50000 (16%)]	Loss: 1957.986206
Train Epoch: 1 [9600/50000 (19%)]	Loss: 2026.598999
Train Epoch: 1 [11200/50000 (22%)]	Loss: 2020.721436
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1991.320068
Train Epoch: 1 [14400/50000 (29%)]	Loss: 1968.313477
Train Epoch: 1 [16000/50000 (32%)]	Loss: 1983.819702
Train Epoch: 1 [17600/50000 (35%)]	Loss: 1921.529297
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1988.078125
Train Epoch: 1 [20800/50000 (42%)]	Loss: 2001.412476
Train Epoch: 1 [22400/50000 (45%)]	Loss: 1941.619873
Train Epoch: 1 [24000/50000 (48%)]	Loss: 1947.984497
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1986.812256
Train Epoch: 1 [27200/50000 (54%)]	Loss: 1972.718018
Train Epoch: 1 [28800/50000 (58%)]	Loss: 1976.627441
Train 

Train Epoch: 2 [44800/50000 (90%)]	Loss: 1902.412231
Train Epoch: 2 [46400/50000 (93%)]	Loss: 1897.885986
Train Epoch: 2 [48000/50000 (96%)]	Loss: 1957.689697
Train Epoch: 2 [49600/50000 (99%)]	Loss: 1900.385986
====> Epoch: 2 Average loss: 1939.1848
Train Epoch: 3 [0/50000 (0%)]	Loss: 1965.103149
Train Epoch: 3 [1600/50000 (3%)]	Loss: 1928.350586
Train Epoch: 3 [3200/50000 (6%)]	Loss: 1968.498779
Train Epoch: 3 [4800/50000 (10%)]	Loss: 1869.194214
Train Epoch: 3 [6400/50000 (13%)]	Loss: 1945.661743
Train Epoch: 3 [8000/50000 (16%)]	Loss: 1953.221069
Train Epoch: 3 [9600/50000 (19%)]	Loss: 1921.662720
Train Epoch: 3 [11200/50000 (22%)]	Loss: 1950.686890
Train Epoch: 3 [12800/50000 (26%)]	Loss: 1909.235229
Train Epoch: 3 [14400/50000 (29%)]	Loss: 1928.724976
Train Epoch: 3 [16000/50000 (32%)]	Loss: 1923.795776
Train Epoch: 3 [17600/50000 (35%)]	Loss: 1938.602661
Train Epoch: 3 [19200/50000 (38%)]	Loss: 1942.563232
Train Epoch: 3 [20800/50000 (42%)]	Loss: 1924.825806
Train Epoch: 3 [2240

Train Epoch: 4 [38400/50000 (77%)]	Loss: 1914.007446
Train Epoch: 4 [40000/50000 (80%)]	Loss: 1931.157104
Train Epoch: 4 [41600/50000 (83%)]	Loss: 1868.746094
Train Epoch: 4 [43200/50000 (86%)]	Loss: 1906.015869
Train Epoch: 4 [44800/50000 (90%)]	Loss: 1913.072144
Train Epoch: 4 [46400/50000 (93%)]	Loss: 1911.126465
Train Epoch: 4 [48000/50000 (96%)]	Loss: 1918.248535
Train Epoch: 4 [49600/50000 (99%)]	Loss: 1914.735352
====> Epoch: 4 Average loss: 1916.4158
Train Epoch: 5 [0/50000 (0%)]	Loss: 1864.680420
Train Epoch: 5 [1600/50000 (3%)]	Loss: 1901.128906
Train Epoch: 5 [3200/50000 (6%)]	Loss: 1961.216675
Train Epoch: 5 [4800/50000 (10%)]	Loss: 1881.862183
Train Epoch: 5 [6400/50000 (13%)]	Loss: 1907.244751
Train Epoch: 5 [8000/50000 (16%)]	Loss: 1888.116943
Train Epoch: 5 [9600/50000 (19%)]	Loss: 1868.255859
Train Epoch: 5 [11200/50000 (22%)]	Loss: 1890.424927
Train Epoch: 5 [12800/50000 (26%)]	Loss: 1898.507324
Train Epoch: 5 [14400/50000 (29%)]	Loss: 1916.878784
Train Epoch: 5 [1600

Train Epoch: 6 [32000/50000 (64%)]	Loss: 1902.894653
Train Epoch: 6 [33600/50000 (67%)]	Loss: 1910.661255
Train Epoch: 6 [35200/50000 (70%)]	Loss: 1868.538696
Train Epoch: 6 [36800/50000 (74%)]	Loss: 1924.565063
Train Epoch: 6 [38400/50000 (77%)]	Loss: 1894.270874
Train Epoch: 6 [40000/50000 (80%)]	Loss: 1822.382690
Train Epoch: 6 [41600/50000 (83%)]	Loss: 1895.246094
Train Epoch: 6 [43200/50000 (86%)]	Loss: 1915.796143
Train Epoch: 6 [44800/50000 (90%)]	Loss: 1873.997803
Train Epoch: 6 [46400/50000 (93%)]	Loss: 1915.083496
Train Epoch: 6 [48000/50000 (96%)]	Loss: 1877.462891
Train Epoch: 6 [49600/50000 (99%)]	Loss: 1884.027588
====> Epoch: 6 Average loss: 1904.7381
Train Epoch: 7 [0/50000 (0%)]	Loss: 1937.737061
Train Epoch: 7 [1600/50000 (3%)]	Loss: 1883.407715
Train Epoch: 7 [3200/50000 (6%)]	Loss: 1922.483276
Train Epoch: 7 [4800/50000 (10%)]	Loss: 1911.778809
Train Epoch: 7 [6400/50000 (13%)]	Loss: 1849.558472
Train Epoch: 7 [8000/50000 (16%)]	Loss: 1991.335449
Train Epoch: 7 [960

Train Epoch: 8 [25600/50000 (51%)]	Loss: 1805.806152
Train Epoch: 8 [27200/50000 (54%)]	Loss: 1925.185547
Train Epoch: 8 [28800/50000 (58%)]	Loss: 1891.230103
Train Epoch: 8 [30400/50000 (61%)]	Loss: 1865.032837
Train Epoch: 8 [32000/50000 (64%)]	Loss: 1868.315674
Train Epoch: 8 [33600/50000 (67%)]	Loss: 1893.211426
Train Epoch: 8 [35200/50000 (70%)]	Loss: 1865.249878
Train Epoch: 8 [36800/50000 (74%)]	Loss: 1904.299683
Train Epoch: 8 [38400/50000 (77%)]	Loss: 1890.601562
Train Epoch: 8 [40000/50000 (80%)]	Loss: 1900.582886
Train Epoch: 8 [41600/50000 (83%)]	Loss: 1897.883301
Train Epoch: 8 [43200/50000 (86%)]	Loss: 1865.590820
Train Epoch: 8 [44800/50000 (90%)]	Loss: 1811.689575
Train Epoch: 8 [46400/50000 (93%)]	Loss: 1880.592529
Train Epoch: 8 [48000/50000 (96%)]	Loss: 1903.489258
Train Epoch: 8 [49600/50000 (99%)]	Loss: 1916.998901
====> Epoch: 8 Average loss: 1901.4407
Train Epoch: 9 [0/50000 (0%)]	Loss: 1862.343750
Train Epoch: 9 [1600/50000 (3%)]	Loss: 1868.554321
Train Epoch: 9

...

Train Epoch: 91 [46400/50000 (93%)]	Loss: 1865.003418
Train Epoch: 91 [48000/50000 (96%)]	Loss: 1831.010254
Train Epoch: 91 [49600/50000 (99%)]	Loss: 1860.414917
====> Epoch: 91 Average loss: 1847.2044
Train Epoch: 92 [0/50000 (0%)]	Loss: 1838.199829
Train Epoch: 92 [1600/50000 (3%)]	Loss: 1842.573730
Train Epoch: 92 [3200/50000 (6%)]	Loss: 1864.116455
Train Epoch: 92 [4800/50000 (10%)]	Loss: 1880.257568
Train Epoch: 92 [6400/50000 (13%)]	Loss: 1840.579224
Train Epoch: 92 [8000/50000 (16%)]	Loss: 1744.070557
Train Epoch: 92 [9600/50000 (19%)]	Loss: 1825.610840
Train Epoch: 92 [11200/50000 (22%)]	Loss: 1891.257690
Train Epoch: 92 [12800/50000 (26%)]	Loss: 1810.416382
Train Epoch: 92 [14400/50000 (29%)]	Loss: 1836.955566
Train Epoch: 92 [16000/50000 (32%)]	Loss: 1936.664551
Train Epoch: 92 [17600/50000 (35%)]	Loss: 1889.766724
Train Epoch: 92 [19200/50000 (38%)]	Loss: 1868.127930
Train Epoch: 92 [20800/50000 (42%)]	Loss: 1826.763062
Train Epoch: 92 [22400/50000 (45%)]	Loss: 1887.387695
T

Train Epoch: 93 [38400/50000 (77%)]	Loss: 1839.223999
Train Epoch: 93 [40000/50000 (80%)]	Loss: 1808.049683
Train Epoch: 93 [41600/50000 (83%)]	Loss: 1854.722534
Train Epoch: 93 [43200/50000 (86%)]	Loss: 1834.017944
Train Epoch: 93 [44800/50000 (90%)]	Loss: 1851.762939
Train Epoch: 93 [46400/50000 (93%)]	Loss: 1885.607056
Train Epoch: 93 [48000/50000 (96%)]	Loss: 1893.729248
Train Epoch: 93 [49600/50000 (99%)]	Loss: 1801.282593
====> Epoch: 93 Average loss: 1846.9187
Train Epoch: 94 [0/50000 (0%)]	Loss: 1859.931519
Train Epoch: 94 [1600/50000 (3%)]	Loss: 1851.029053
Train Epoch: 94 [3200/50000 (6%)]	Loss: 1878.809570
Train Epoch: 94 [4800/50000 (10%)]	Loss: 1746.875732
Train Epoch: 94 [6400/50000 (13%)]	Loss: 1845.105103
Train Epoch: 94 [8000/50000 (16%)]	Loss: 1866.681519
Train Epoch: 94 [9600/50000 (19%)]	Loss: 1921.218994
Train Epoch: 94 [11200/50000 (22%)]	Loss: 1859.772095
Train Epoch: 94 [12800/50000 (26%)]	Loss: 1817.284668
Train Epoch: 94 [14400/50000 (29%)]	Loss: 1904.098999
T

Train Epoch: 95 [30400/50000 (61%)]	Loss: 1851.400391
Train Epoch: 95 [32000/50000 (64%)]	Loss: 1855.115845
Train Epoch: 95 [33600/50000 (67%)]	Loss: 1880.600342
Train Epoch: 95 [35200/50000 (70%)]	Loss: 1718.274170
Train Epoch: 95 [36800/50000 (74%)]	Loss: 1877.428101
Train Epoch: 95 [38400/50000 (77%)]	Loss: 1814.153442
Train Epoch: 95 [40000/50000 (80%)]	Loss: 1709.736450
Train Epoch: 95 [41600/50000 (83%)]	Loss: 1885.552612
Train Epoch: 95 [43200/50000 (86%)]	Loss: 1819.214478
Train Epoch: 95 [44800/50000 (90%)]	Loss: 1835.534546
Train Epoch: 95 [46400/50000 (93%)]	Loss: 1775.114502
Train Epoch: 95 [48000/50000 (96%)]	Loss: 1876.278076
Train Epoch: 95 [49600/50000 (99%)]	Loss: 1899.321167
====> Epoch: 95 Average loss: 1846.5767
Train Epoch: 96 [0/50000 (0%)]	Loss: 1859.859619
Train Epoch: 96 [1600/50000 (3%)]	Loss: 1703.162109
Train Epoch: 96 [3200/50000 (6%)]	Loss: 1801.199341
Train Epoch: 96 [4800/50000 (10%)]	Loss: 1827.189575
Train Epoch: 96 [6400/50000 (13%)]	Loss: 1838.878296

Train Epoch: 97 [22400/50000 (45%)]	Loss: 1886.563843
Train Epoch: 97 [24000/50000 (48%)]	Loss: 1880.268677
Train Epoch: 97 [25600/50000 (51%)]	Loss: 1849.800903
Train Epoch: 97 [27200/50000 (54%)]	Loss: 1861.184204
Train Epoch: 97 [28800/50000 (58%)]	Loss: 1894.944580
Train Epoch: 97 [30400/50000 (61%)]	Loss: 1823.979736
Train Epoch: 97 [32000/50000 (64%)]	Loss: 1882.014282
Train Epoch: 97 [33600/50000 (67%)]	Loss: 1872.525513
Train Epoch: 97 [35200/50000 (70%)]	Loss: 1846.765869
Train Epoch: 97 [36800/50000 (74%)]	Loss: 1874.178101
Train Epoch: 97 [38400/50000 (77%)]	Loss: 1764.657837
Train Epoch: 97 [40000/50000 (80%)]	Loss: 1886.916992
Train Epoch: 97 [41600/50000 (83%)]	Loss: 1834.891602
Train Epoch: 97 [43200/50000 (86%)]	Loss: 1884.797241
Train Epoch: 97 [44800/50000 (90%)]	Loss: 1872.937622
Train Epoch: 97 [46400/50000 (93%)]	Loss: 1846.396240
Train Epoch: 97 [48000/50000 (96%)]	Loss: 1893.675415
Train Epoch: 97 [49600/50000 (99%)]	Loss: 1855.102783
====> Epoch: 97 Average loss

Train Epoch: 99 [14400/50000 (29%)]	Loss: 1782.653809
Train Epoch: 99 [16000/50000 (32%)]	Loss: 1819.945923
Train Epoch: 99 [17600/50000 (35%)]	Loss: 1837.811890
Train Epoch: 99 [19200/50000 (38%)]	Loss: 1784.092773
Train Epoch: 99 [20800/50000 (42%)]	Loss: 1880.887207
Train Epoch: 99 [22400/50000 (45%)]	Loss: 1757.744263
Train Epoch: 99 [24000/50000 (48%)]	Loss: 1858.514648
Train Epoch: 99 [25600/50000 (51%)]	Loss: 1829.080566
Train Epoch: 99 [27200/50000 (54%)]	Loss: 1830.883179
Train Epoch: 99 [28800/50000 (58%)]	Loss: 1875.606567
Train Epoch: 99 [30400/50000 (61%)]	Loss: 1828.478394
Train Epoch: 99 [32000/50000 (64%)]	Loss: 1842.704590
Train Epoch: 99 [33600/50000 (67%)]	Loss: 1803.596802
Train Epoch: 99 [35200/50000 (70%)]	Loss: 1905.246582
Train Epoch: 99 [36800/50000 (74%)]	Loss: 1859.072021
Train Epoch: 99 [38400/50000 (77%)]	Loss: 1821.467529
Train Epoch: 99 [40000/50000 (80%)]	Loss: 1878.802979
Train Epoch: 99 [41600/50000 (83%)]	Loss: 1861.789551
Train Epoch: 99 [43200/50000

In [0]:
with torch.no_grad():
        sample = torch.randn(64, 32).cuda()
        sample = model.decode(sample)
        torch.save(model.cpu().state_dict(), "./save_checkpoint_epoch_"+str(epoch)+".pth")
        files.download("./save_checkpoint_epoch_"+str(epoch)+".pth")
        torchvision.utils.save_image(sample.view(64, 3, 32, 32),'./sample_' + str(epoch) + '.png')
        files.download('./sample_' + str(epoch) + '.png')